In [8]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import face_recognition
import cv2
import tkinter as tk
from tkinter import Button, Label, Frame
import datetime
from PIL import Image, ImageTk
import threading

### Intento

In [2]:
import os
import face_recognition
import cv2
import tkinter as tk
from tkinter import Button, Label, Frame
import datetime
from PIL import Image, ImageTk

# Obtener la fecha actual
fecha_actual = datetime.datetime.now().strftime('%Y-%m-%d')
carpeta_registro = f"registro_{fecha_actual}"

# Lista para almacenar las codificaciones faciales de las personas de referencia y sus nombres asociados
personas_codificaciones = []
nombres = []

# Cargar imágenes de referencia y calcular codificaciones faciales para cada persona
nombres_archivos = ["pablo.jpg", "gabriel.jpg", "miguel.jpg", "richard.jpg", "alejandra.jpg", "ariane.jpg", "camila.jpg", "camilo.jpg", "dylanJ.jpg", "hugo.jpg", "ignacio.jpg", "jose.jpg", "joseph.jpg", "marcos.jpg", "mauricio.jpg", "samuel.jpg", "sebastian.jpg"]
for nombre_archivo in nombres_archivos:
    imagen_referencia = face_recognition.load_image_file(nombre_archivo)
    codificacion = face_recognition.face_encodings(imagen_referencia)

    if len(codificacion) > 0:
        personas_codificaciones.append(codificacion[0])
        nombres.append(nombre_archivo.split(".")[0])

# Crear una carpeta de registro con la fecha actual si no existe
if not os.path.exists(carpeta_registro):
    os.makedirs(carpeta_registro)

# Función para iniciar el reconocimiento facial
def iniciar_reconocimiento():
    global ultima_captura
    # Actualizar el mensaje en la interfaz gráfica
    lbl_estado.config(text="Empezando reconocimiento...")

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    while True:
        r, frame = cap.read()
        face_actual = face_recognition.face_encodings(frame)

        if len(face_actual) > 0:
            resultados = face_recognition.compare_faces(personas_codificaciones, face_actual[0])
            nombre_persona = "Desconocido"

            for i, resultado in enumerate(resultados):
                if resultado:
                    nombre_persona = nombres[i]
                    break

            # Actualizar el mensaje en la interfaz gráfica
            lbl_estado.config(text=f"Persona reconocida: {nombre_persona}")

            # Dibujar rectángulo y mostrar el nombre de la persona reconocida
            face_loc = face_recognition.face_locations(frame)
            for loc in face_loc:
                cv2.rectangle(frame, (loc[3], loc[0]), (loc[1], loc[2]), (0, 255, 0), 5)
                cv2.putText(frame, nombre_persona, (loc[3], loc[2] + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Guardar la captura en la carpeta de registro
            ahora = datetime.datetime.now()
            nombre_archivo = f"{carpeta_registro}/{nombre_persona}_{ahora.strftime('%Y-%m-%d_%H-%M-%S')}.jpg"
            cv2.imwrite(nombre_archivo, frame)

            # Convertir la imagen a formato PIL
            ultima_captura = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            ultima_captura = ultima_captura.resize((400, 300), Image.ANTIALIAS)  # Cambia el tamaño según tus necesidades

            # Mostrar la última captura en la interfaz gráfica
            imagen = ImageTk.PhotoImage(ultima_captura)
            last_capture_label.config(image=imagen)
            last_capture_label.image = imagen

        cv2.imshow('Reconocimiento Facial', frame)

        wait_key = cv2.waitKey(1)
        if wait_key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

# Crear la interfaz gráfica
root = tk.Tk()
root.title("Sistema de Reconocimiento Facial")

# Configurar el tamaño de la ventana
root.geometry("800x600")
root.configure(bg="#f0f0f0")  # Color de fondo

# Texto de bienvenida
lbl_bienvenido = tk.Label(root, text="Bienvenido!", font=("Arial", 36), bg="#f0f0f0", fg="#333")  # Cambia los colores de fondo y texto
lbl_bienvenido.pack(pady=20)

# Marco para la última captura
frame_ultima_captura = Frame(root, bg="#f0f0f0")
frame_ultima_captura.pack(pady=20)

# Etiqueta para mostrar la última captura
last_capture_label = Label(frame_ultima_captura)
last_capture_label.pack()

# Botón para iniciar el reconocimiento facial
btn_iniciar = Button(root, text="Iniciar Reconocimiento", font=("Arial", 18), command=iniciar_reconocimiento, bg="#4caf50", fg="white")  # Cambia el color de fondo y texto del botón
btn_iniciar.pack(pady=20)

# Etiqueta para mostrar el estado del reconocimiento
lbl_estado = Label(root, text="Esperando reconocimiento...", font=("Arial", 24), bg="#f0f0f0", fg="#333")  # Cambia los colores de fondo y texto
lbl_estado.pack()

root.mainloop()


C:\Users\gabon\AppData\Local\Temp\ipykernel_7296\3567251597.py:70: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  ultima_captura = ultima_captura.resize((400, 300), Image.ANTIALIAS)  # Cambia el tamaño según tus necesidades


### Threading
Esto lo hacemos para evitar el hecho de que las operaciones de captura y procesamiento de imágenes están ocurriendo en el hilo principal de la interfaz gráfica, lo que puede afectar su capacidad para responder rápidamente a las actualizaciones.

Para solucionar esto, movimos las operaciones de captura y procesamiento de imágenes a un hilo separado utilizando el módulo threading. Esto permitirá que la interfaz gráfica se actualice de forma más rápida, ya que las operaciones pesadas se realizarán en segundo plano.

In [2]:
# Obtener la fecha actual
fecha_actual = datetime.datetime.now().strftime('%Y-%m-%d')
carpeta_registro = f"registro_{fecha_actual}"

# Lista para almacenar las codificaciones faciales de las personas de referencia y sus nombres asociados
personas_codificaciones = []
nombres = []

# Cargar imágenes de referencia y calcular codificaciones faciales para cada persona
nombres_archivos = ["pablo.jpg", "gabriel.jpg", "miguel.jpg", "richard.jpg", "alejandra.jpg", "ariane.jpg", "camila.jpg", "camilo.jpg", "dylanJ.jpg", "hugo.jpg", "ignacio.jpg", "jose.jpg", "joseph.jpg", "marcos.jpg", "mauricio.jpg", "samuel.jpg", "sebastian.jpg"]
for nombre_archivo in nombres_archivos:
    imagen_referencia = face_recognition.load_image_file(nombre_archivo)
    codificacion = face_recognition.face_encodings(imagen_referencia)

    if len(codificacion) > 0:
        personas_codificaciones.append(codificacion[0])
        nombres.append(nombre_archivo.split(".")[0])

# Crear una carpeta de registro con la fecha actual si no existe
if not os.path.exists(carpeta_registro):
    os.makedirs(carpeta_registro)

# Función para iniciar el reconocimiento facial
def iniciar_reconocimiento():
    # Actualizar el mensaje en la interfaz gráfica
    lbl_estado.config(text="Empezando reconocimiento...")
    
    def reconocimiento():
        global ultima_captura

        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

        while True:
            r, frame = cap.read()
            face_actual = face_recognition.face_encodings(frame)

            if len(face_actual) > 0:
                resultados = face_recognition.compare_faces(personas_codificaciones, face_actual[0])
                nombre_persona = "Desconocido"

                for i, resultado in enumerate(resultados):
                    if resultado:
                        nombre_persona = nombres[i]
                        break

                # Actualizar el mensaje en la interfaz gráfica
                lbl_estado.config(text=f"Persona reconocida: {nombre_persona}")

                # Dibujar rectángulo y mostrar el nombre de la persona reconocida
                face_loc = face_recognition.face_locations(frame)
                for loc in face_loc:
                    cv2.rectangle(frame, (loc[3], loc[0]), (loc[1], loc[2]), (0, 255, 0), 5)
                    cv2.putText(frame, nombre_persona, (loc[3], loc[2] + 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Guardar la captura en la carpeta de registro
                ahora = datetime.datetime.now()
                nombre_archivo = f"{carpeta_registro}/{nombre_persona}_{ahora.strftime('%Y-%m-%d_%H-%M-%S')}.jpg"
                cv2.imwrite(nombre_archivo, frame)

                # Convertir la imagen a formato PIL
                ultima_captura = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                ultima_captura = ultima_captura.resize((400, 300), Image.ANTIALIAS)  # Cambia el tamaño según tus necesidades

                # Mostrar la última captura en la interfaz gráfica
                imagen = ImageTk.PhotoImage(ultima_captura)
                last_capture_label.config(image=imagen)
                last_capture_label.image = imagen

            cv2.imshow('Reconocimiento Facial', frame)

            wait_key = cv2.waitKey(1)
            if wait_key == 27:
                break

        cap.release()
        cv2.destroyAllWindows()

    # Iniciar un hilo para realizar el reconocimiento facial
    threading.Thread(target=reconocimiento).start()

# Crear la interfaz gráfica
root = tk.Tk()
root.title("Sistema de Reconocimiento Facial")

# Configurar el tamaño de la ventana
root.geometry("800x600")
root.configure(bg="#f0f0f0")  # Color de fondo

# Texto de bienvenida
lbl_bienvenido = tk.Label(root, text="Bienvenido!", font=("Arial", 36), bg="#f0f0f0", fg="#333")  # Cambia los colores de fondo y texto
lbl_bienvenido.pack(pady=20)

# Marco para la última captura
frame_ultima_captura = Frame(root, bg="#f0f0f0")
frame_ultima_captura.pack(pady=20)

# Etiqueta para mostrar la última captura
last_capture_label = Label(frame_ultima_captura)
last_capture_label.pack()

# Botón para iniciar el reconocimiento facial
btn_iniciar = Button(root, text="Iniciar Reconocimiento", font=("Arial", 18), command=iniciar_reconocimiento, bg="#4caf50", fg="white")  # Cambia el color de fondo y texto del botón
btn_iniciar.pack(pady=20)

# Etiqueta para mostrar el estado del reconocimiento
lbl_estado = Label(root, text="Esperando reconocimiento...", font=("Arial", 24), bg="#f0f0f0", fg="#333")  # Cambia los colores de fondo y texto
lbl_estado.pack()

root.mainloop()


C:\Users\gabon\AppData\Local\Temp\ipykernel_18412\448845635.py:64: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  ultima_captura = ultima_captura.resize((400, 300), Image.ANTIALIAS)  # Cambia el tamaño según tus necesidades
